<a href="https://colab.research.google.com/github/kentokura/dmps_tournament_assist/blob/master/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# What's notebook?

このノートは、アプリ「[DUEL MASTERS PLAY’S（デュエル・マスターズ プレイス）](https://tonamel.com/competition/hi9my)」の特殊レギュレーション大会を開催するにあたり、主催者が行わなければならない大量のデッキの確認を自動化してくれます。レギュレーションの対応は検討中です。<!-- BASIC+40, BASIC+60, ハイランダーに対応しています。 -->

## Tonamelについて

[Tonamel](https://tonamel.com/)を使えば、誰でも簡単に大会を開催・運営できます。エントリーから大会終了までに必要な機能が揃っています。

特に、主催者は参加者情報をcsv形式でダウンロードでき、デュエプレ公式ページからデッキURLを作成して申し込み時に入力してもらうことで、**参加者とそのデッキの情報をcsv形式で取得**することができます。


## 大会ひな型
---

【デュエプレ　特殊レギュレーション大会】
BASIC+60_β杯を開催します。
以下の説明を読んで、応募してください。

# BASIC+60とは？
カードの各レアリティにポイントが割り振られています。 60ポイント以内に収まるようにデッキを構築し、対戦します。
ポイントは、以下の通りです:
- レア度 ... 1枚当たりのポイント
- BASIC ... 0   p/1枚
- C .......... 2   p/1枚
- UC ........ 4   p/1枚
- R........... 8   p/1枚
- VR ........ 12 p/1枚
- SR ........ 20 p/1枚

また、一部カードはレア度変更してカウントいたします。
- 調整カード名 ............ レア度の扱い
- ジェノサイドワーム ...  BASIC → C
- クエイクゲート .........  C → UC

# デッキ提出
デュエプレ公式サイト(https://dmps.takaratomy.co.jp/deckbuilder/create/)
からデッキのURLを作成してください。
エントリー時に提出します。

# 賞金
ないよ。

# 注意事項

参加される方は、以下を全て読んでください。

・20:55までには主の配信(https://www.mirrativ.com/user/99939804)に参加してください。
・開始時刻までに来ない場合やデッキの確認が取れない場合、
　その他運営の進行を妨げると判断した場合には参加をキャンセルする場合があります。
・勝敗について虚偽の報告がある場合、それを嘘だと判断する証拠がない場合は、
　両者失格・キックする場合がございます。勝者は勝敗が確定時のスクリーンショットを
　取っておくことをお勧めいたします。
・デッキは後日公開させていただく場合がございます。あらかじめご了承ください。
・参加をキャンセルされる場合は、なるべく早くご連絡お願いいたします

---

# やりたいこと

[Tonamel](https://tonamel.com/)からダウンロードできる参加者情報のcsvファイルを入力し、デッキ確認結果を出力する。


# 処理(モックアップ)

## 準備

### [1] GDriveをマウントする。

ここからカード情報のcsvを読み込む。



In [2]:
# driveのマウント
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


### [2] **Deckクラス**の作成

コードの可読性を上げるためにクラスを作る。

In [0]:
class Deck():
    """
    デッキ内のカード情報やヘルパー関数を保持する。

    Attributes
    ----------
    cards : list
        カード情報。
    """


    def __init__(self):
        """
        Parameters
        ----------
        cards : list
            カード情報。
        card_num : int
            デッキ内のカード枚数
        """
        self.cards = {} # {id : detail=[num, ...]}
        self.card_num = 0

    def set_card(self, detail):
        """
        カード情報をセットする。

        Parameters
        ----------
        num : int
            カード枚数。
        detail : ndarray
            カード詳細。
        """
        self.cards[detail[0]] = detail[1:]
        self.card_num += detail[1]

    def print_all(self):
        """
        デッキの内容を出力する。
        """
        print("カード枚数: " + str(self.card_num))
        for card in self.cards.values():
            print(card)

### [3] 便利な関数

- **encode_cardid_by_url(url: str) -> dict**の作成

  URLの前処理を行う。

In [0]:
def encode_cardid_by_url(url: str) -> dict: 
    """
    URLから、カードidごとの枚数の辞書を作成する。

    Parameters
    ----------
    url : str
        デッキURL。
    
    Returns
    -------
    deck : dict of { str, int }
        カードの種類ごとの枚数。    
    """
    try:
        # 型の確認
        if type(url) != type("string"):
            raise Exception("URLは文字列で指定してください")

        # URLからカードidを取得
        site_url, card_url = url.split("c=")
        card_url, key_card_url = card_url.split("&")
        arr_card_id = card_url.split(".")

        # 40枚なかったらエラーを返す
        if len(arr_card_id) != 40:
            raise Exception("カードが40枚ありません")
        
        deck = { card_id: arr_card_id.count(card_id) for card_id in arr_card_id }
    
        return deck

    except Exception as e:
        print("エラーが発生しました")
        print(e)


## 入力

1. csvを読み込む
1. URLからcardidごとの枚数を取得
1. 取得したデータとcsvからデッキ情報を作成

In [5]:
import pandas as pd
from pandas import DataFrame
import numpy as np

# csvの読み込み
dmps_card_db = pd.read_csv(
    "/content/gdrive/My Drive/card_csv/dmps_card_db.csv",
    encoding="cp932",
    )

# URLからdeck{'id':'card_num'}を取得
deck_about = encode_cardid_by_url(
    "https://dmps.takaratomy.co.jp/deckbuilder/deck/" + 
    "?c=DEAJ.DEAJ.DEAJ.DEAJ.DE99.DE99.DE99.DE99.MQAK.MQAK" + 
    ".MQAK.MQAK.PUAN.PUAN.PUAN.PUAN.4EBF.4EBF.4EBF.4EBF" + 
    ".JMAB.JMAB.JMAB.JMAB.4EBU.4EBU.4EBU.4EBU.AAAS.AAAS.AAAS"+
    ".AAAS.4EAR.4EAR.4EAR.4EAR.DEAT.DEAT.DEAT.DEAT&k=DEAJ"
    )#input())

# Deckにまとめる
deck = Deck()
for card_id, num in deck_about.items():
    try:
        # 一致する行をndarray型で取得
        # Todo: 各要素の先頭になぜか空白が入る問題の修正
        card_detail = dmps_card_db.query("id==@card_id").to_numpy(copy=True).flatten()
        if len(card_detail) == 0: # dbと一致しなかった
            raise Exception(card_id + "と一致するカードがdbにありません。")
        else: # 正常系
            card = np.insert(arr=card_detail, obj=1, values=num)
            deck.set_card(card)
    except Exception as e:
        print("エラーが発生しました")
        print(e)
        print()

# 結果出力
# deck.print_all()

# for card_detail in card_details:
#     print(card_detail)

エラーが発生しました
DE99と一致するカードがdbにありません。



## 出力テスト

ちゃんと入力がとれているか確認


In [6]:
deck.print_all()

カード枚数: 36
[4 '予言者ジェス' ' ライトブリンガー' ' ' ' C' 1 ' 光' ' ' ' クリーチャー']
[4 'プロテクト・フォース' ' 呪文' ' ' ' C' 1 ' 光' ' ' ' 呪文']
[4 'ゼピメテウス' ' シー・ハッカー' ' ' ' C' 1 ' 水' ' ' ' クリーチャー']
[4 'トリア' ' サイバーロード' ' ' ' C' 1 ' 水' ' ' ' クリーチャー']
[4 'アクア・ガード' ' リキッド・ピープル' ' ' ' BASIC' 1 ' 水' ' ' ' クリーチャー']
[4 '孤独の影ロンリー・ウォーカー' ' ゴースト' ' ' ' UC' 1 ' 闇' ' ' ' クリーチャー']
[4 'ねじれる者ボーン・スライム' ' リビング・デッド' ' ' ' C' 1 ' 闇' ' ' ' クリーチャー']
[4 '暴虐虫タイラント・ワーム' ' パラサイトワーム' ' ' ' C' 1 ' 闇' ' ' ' クリーチャー']
[4 'ブラック・スレイヤー' ' 呪文' ' ' ' C' 1 ' 闇' ' ' ' 呪文']


## BASIC+60バリデート

レギュレーションに沿っているか確認する。
デッキの合計ポイントと合否を出力する。

In [7]:
# ポイント上限
point_max = 60

# カード枚数の確認
print("カード枚数\t: ",deck.card_num)

# ポイントの確認
total_point = 0
for card in deck.cards.values():
    point = 0
    # 空白が入っていることに注意
    if card[4] == ' BASIC':
        point = 0
    elif card[4] == ' C':
        point = 2
    elif card[4] == ' UC':
        point = 4
    elif card[4] == ' R':
        point = 4
    elif card[4] == ' VR':
        point = 4
    elif card[4] == ' SR':
        point = 4
    else:
        print("レア度が不正です。")
    
    total_point += point*card[0]

print("合計ポイント\t: ",total_point)

if total_point > point_max:
    print("判定\t\t: ","OUT")
else:
    print("判定\t\t: ","OK")

カード枚数	:  36
合計ポイント	:  72
判定		:  OUT


# 処理(ガチ)

## 準備

### 必要なデータをダウンロード
[こちら](https://github.com/kentokura/dmps_tournament_assist)のリポジトリをクローンします。主に使用するのは以下のファイルです。

| ファイル名 | 説明 |
| --- | --- |
| dmps_cards | カードセットがいくつかのcsvファイルに分かれてあります。|
| src | こちらで使う関数が定義されています。 |


In [8]:
# リポジトリをクローンする
!git clone https://github.com/kentokura/dmps_tournament_assist
# 確認
!ls dmps_tournament_assist/

Cloning into 'dmps_tournament_assist'...
remote: Enumerating objects: 165, done.
remote: Counting objects: 100% (165/165), done.
remote: Compressing objects: 100% (119/119), done.
remote: Total 165 (delta 43), reused 127 (delta 29), pack-reused 0
Receiving objects: 100% (165/165), 494.84 KiB | 10.31 MiB/s, done.
Resolving deltas: 100% (43/43), done.
dmps_cards  LICENSE  main.ipynb  README.md  src


### [1] GDriveをマウントする。

ここからカード情報のcsvを読み込む。



In [9]:
# driveのマウント
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


### [2] **Deckクラス**の作成

コードの可読性を上げるためにクラスを作る。

In [0]:
class Deck():
    """
    デッキ内のカード情報やヘルパー関数を保持する。

    Attributes
    ----------
    cards : list
        カード情報。
    card_num : int
        デッキ内のカード枚数。
    regulation : str
        大会レギュレーション。
    is_pass_validate : bool
        regulationに沿った検査の結果。

    """


    def __init__(self, regulation='Nomal'):
        """
        Parameters
        ----------
        cards : list
            カード情報。
        card_num : int
            デッキ内のカード枚数。
        regulation : str
            大会レギュレーション。
        is_pass_validate : bool
            regulationに沿った検査の結果。
        """
        self.cards = {} # {id : detail=[num, ...]}
        self.card_num = 0
        self.regulation = regulation
        self.is_pass_validate = False

    def set_card(self, detail):
        """
        カード情報をセットする。

        Parameters
        ----------
        num : int
            カード枚数。
        detail : ndarray
            カード詳細。
        """
        self.cards[detail[0]] = detail[1:]
        self.card_num += detail[1]

    def print_all(self):
        """
        デッキの内容を出力する。
        """
        print("カード枚数: " + str(self.card_num))
        for card in self.cards.values():
            print(card)
    
    def validate(self):
        """
        確認する。
        """
        if self.regulation == 'Nomal':
            self.is_pass_validate = True
        elif self.regulation == 'BASIC+60':
            # ポイント上限
            point_max = 60

            # カード枚数の確認
            # print("カード枚数\t: ",self.card_num)

            # ポイントの確認
            total_point = 0
            for card in self.cards.values():
                point = 0
                # 空白が入っていることに注意
                if card[4] == ' BASIC':
                    point = 0
                elif card[4] == ' C':
                    point = 2
                elif card[4] == ' UC':
                    point = 4
                elif card[4] == ' R':
                    point = 4
                elif card[4] == ' VR':
                    point = 4
                elif card[4] == ' SR':
                    point = 4
                else:
                    print("レア度が不正です。")
                
                total_point += point*card[0]

            # print("合計ポイント\t: ",total_point)

            if total_point > point_max:
                # print("判定\t\t: ","OUT")
                self.is_pass_validate == False
            else:
                # print("判定\t\t: ","OK")
                self.is_pass_validate == True

### [3] 便利な関数

- **encode_cardid_by_url(url: str) -> dict**の作成

  URLの前処理を行う。

In [0]:
def encode_cardid_by_url(url: str) -> dict: 
    """
    URLから、カードidごとの枚数の辞書を作成する。

    Parameters
    ----------
    url : str
        デッキURL。
    
    Returns
    -------
    deck : dict of { str, int }
        カードの種類ごとの枚数。    
    """
    try:
        # 型の確認
        if type(url) != type("string"):
            raise Exception("URLは文字列で指定してください")

        # URLからカードidを取得
        site_url, card_url = url.split("c=")
        card_url, key_card_url = card_url.split("&")
        arr_card_id = card_url.split(".")

        # 40枚なかったらエラーを返す
        if len(arr_card_id) != 40:
            raise Exception("カードが40枚ありません")
        
        deck = { card_id: arr_card_id.count(card_id) for card_id in arr_card_id }
    
        return deck

    except Exception as e:
        print("エラーが発生しました")
        print(e)


## 入力

1. csvを読み込む
1. URLからcardidごとの枚数を取得
1. 取得したデータとcsvからデッキ情報を作成

In [12]:
!ls 

dmps_tournament_assist	gdrive	sample_data


In [13]:
import pandas as pd
from pandas import DataFrame
import numpy as np

# csvの読み込み
dmps_card_db = pd.read_csv(
    "/content/gdrive/My Drive/card_csv/dmps_card_db.csv",
    encoding="cp932",
    )
print(type(dmps_card_db))

# URLからdeck{'id':'card_num'}を取得
deck_about = encode_cardid_by_url(input())
    # "https://dmps.takaratomy.co.jp/deckbuilder/deck/" + 
    # "?c=DEAJ.DEAJ.DEAJ.DEAJ.DE99.DE99.DE99.DE99.MQAK.MQAK" + 
    # ".MQAK.MQAK.PUAN.PUAN.PUAN.PUAN.4EBF.4EBF.4EBF.4EBF" + 
    # ".JMAB.JMAB.JMAB.JMAB.4EBU.4EBU.4EBU.4EBU.AAAS.AAAS.AAAS"+
    # ".AAAS.4EAR.4EAR.4EAR.4EAR.DEAT.DEAT.DEAT.DEAT&k=DEAJ"
    # )#input())

# Deckにまとめる
deck = Deck(regulation="BASIC+60")
for card_id, num in deck_about.items():
    try:
        # 一致する行をndarray型で取得
        # Todo: 各要素の先頭になぜか空白が入る問題の修正
        card_detail = dmps_card_db.query("id==@card_id").to_numpy(copy=True).flatten()
        if len(card_detail) == 0: # dbと一致しなかった
            raise Exception(card_id + "と一致するカードがdbにありません。")
        else: # 正常系
            card = np.insert(arr=card_detail, obj=1, values=num)
            deck.set_card(card)
    except Exception as e:
        print("エラーが発生しました")
        print(e)
        print()

# 結果出力
# deck.print_all()

# for card_detail in card_details:
#     print(card_detail)

<class 'pandas.core.frame.DataFrame'>


KeyboardInterrupt: ignored

## 出力テスト

ちゃんと入力がとれているか確認


In [0]:
deck.print_all()

## BASIC+60バリデート

レギュレーションに沿っているか確認する。
デッキの合計ポイントと合否を出力する。

In [0]:
# dmpsカードリストの読み込み
dmps_card_db = pd.read_csv(
    "/content/gdrive/My Drive/card_csv/dmps_card_db.csv",
    encoding="cp932",
    )

# 参加者一覧の読み込み
from google.colab import files
print("参加者一覧のcsvファイルをアップロードしてください。")
file_name = [ key for key in files.upload().keys()][0]
tonamel_participants = pd.read_csv(file_name)

# 参加者一人ずつ確認していく
for index, row in tonamel_participants.iterrows():

    # URLからdeck{'id':'card_num'}を取得
    num_per_cardid = encode_cardid_by_url(row["デッキURL"])

    # Deckにまとめる
    deck = Deck(regulation="BASIC+60")
    for card_id, num in num_per_cardid.items():
        try:
            # 一致する行をndarray型で取得
            # Todo: 各要素の先頭になぜか空白が入る問題の修正
            card_detail = dmps_card_db.query("id==@card_id").to_numpy(copy=True).flatten()
            if len(card_detail) == 0: # dbと一致しなかった
                raise Exception(card_id + "と一致するカードがdbにありません。")
            else: # 正常系
                card = np.insert(arr=card_detail, obj=1, values=num)
                deck.set_card(card)
        except Exception as e:
            print("エラーが発生しました")
            print(e)
            print()

    # 結果出力
    # 誰？
    print(index)
    print("アカウント名\t= ", row["アカウント名"])
    print("デッキURL\t= ", row["デッキURL"])
    # 情報
    deck.is_pass_validate
    deck.print_all()
    print()

ファイル名を取得

In [0]:
import glob
import pandas as pd
glob.glob('dmps_tournament_assist/dmps_cards/*/*.csv')

dmps_cards = pd.DataFrame()
for card_set in glob.glob('dmps_tournament_assist/dmps_cards/*/*.csv'):
  dmps_cards = pd.read_csv(card_set, encoding="cp932")
  dmps_cards = dmps_cards[['id', 'name', 'cost', 'series']]
  # print(dmps_cards.head()
  print(dmps_cards.to_html())

---
# やり直し

## カード情報csvの読み込み

### githubのリポジトリごとcolabに持ってくる
[こちら](https://github.com/kentokura/dmps_tournament_assist)のリポジトリをクローンします。主に使用するのは以下のファイルです。

| ファイル名 | 説明 |
| --- | --- |
| dmps_cards | カードセットがいくつかのcsvファイルに分かれてあります。|
| src | こちらで使う関数が定義されています。 |


In [1]:
# リポジトリをクローンする
!git clone https://github.com/kentokura/dmps_tournament_assist
# 確認
!ls dmps_tournament_assist/

Cloning into 'dmps_tournament_assist'...
remote: Enumerating objects: 165, done.
remote: Counting objects: 100% (165/165), done.
remote: Compressing objects: 100% (119/119), done.
remote: Total 165 (delta 43), reused 127 (delta 29), pack-reused 0
Receiving objects: 100% (165/165), 494.84 KiB | 5.21 MiB/s, done.
Resolving deltas: 100% (43/43), done.
dmps_cards  LICENSE  main.ipynb  README.md  src


### すべてのcsvを一つのDataFrameにする。

In [2]:
# pd.DataFrame型のdmps_cardsを作成する。
import glob
import pandas as pd
# ファイルパスの取得
glob.glob('dmps_tournament_assist/dmps_cards/*/*.csv')

# すべてのcsvファイルを読み込む
card_set_list = []
for card_set in glob.glob('dmps_tournament_assist/dmps_cards/*/*.csv'):
  card_set_df = pd.read_csv(card_set, encoding="cp932", index_col=0)
  card_set_list.append(card_set_df)
dmps_cards = pd.concat(card_set_list, sort=False)

# ほしい列だけ取り出す
dmps_cards = dmps_cards[['name', 'cost', 'series']]
dmps_cards.head(5)

,name,cost,series
id,,,
DYAP,悪魔神バロム,8,超獣の始動
WQAK,クリスタル・パラディン,4,超獣の始動
ZUBJ,幻想妖精カチュア,5,伝説の再誕
NEAG,精霊王アルカディアス,6,超獣の始動
AUAY,大勇者「ふたつ牙」,6,超獣の始動


## deckURLを渡したらその情報のpd.dataframeを作成してくれる。

現在:

pd.dataframe型のdmps_cardsにすべての情報が入力されている状況

In [3]:
import pandas as pd
from pandas import DataFrame
import numpy as np

# URLからdeck{'id':'card_num'}を取得
dict_deck_about = encode_cardid_by_url(
    "https://dmps.takaratomy.co.jp/deckbuilder/deck/" + 
    "?c=DEAJ.DEAJ.DEAJ.DEAJ.DE99.DE99.DE99.DE99.MQAK.MQAK" + 
    ".MQAK.MQAK.PUAN.PUAN.PUAN.PUAN.4EBF.4EBF.4EBF.4EBF" + 
    ".JMAB.JMAB.JMAB.JMAB.4EBU.4EBU.4EBU.4EBU.AAAS.AAAS.AAAS"+
    ".AAAS.4EAR.4EAR.4EAR.4EAR.DEAT.DEAT.DEAT.DEAT&k=DEAJ") #input())

df_deck_about = pd.DataFrame(dict_deck_about, index=['id'])
print(df_deck_about)

# pd.dataframe型のdeckを作成する
for card_id, num in deck_about.items():
    try:
        # 一致する行をndarray型で取得
        # Todo: 各要素の先頭になぜか空白が入る問題の修正
        card_detail = dmps_cards.query("id==@card_id").to_numpy(copy=True).flatten()
        if len(card_detail) == 0: # dbと一致しなかった
            raise Exception(card_id + "と一致するカードがdbにありません。")
        else: # 正常系
            card = np.insert(arr=card_detail, obj=1, values=num)
            deck.set_card(card)
    except Exception as e:
        print("エラーが発生しました")
        print(e)
        print()

# 結果出力

NameError: ignored